In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('sales.db')
c = conn.cursor()

In [2]:
####################################### Report 1 #############################################
report1_sql = '''select temperature_master.avg_sf_temperature, sales_2016.item_name, sum(sales_2016.net_quantity)
                 from sales_2016 join temperature_master on sales_2016.sale_date = temperature_master.date
                 group by sales_2016.item_name 
                 order by sum(sales_2016.net_quantity) desc, sales_2016.item_name'''
c.execute(report1_sql)
print(c.fetchall())

[(56, 'Latte', 1945), (68, 'New Orleans Iced Coffee (cup)', 1913), (61, 'Drip Coffee', 1730), (65, 'Cappuccino, Unknown', 1494), (70, 'Mocha', 444), (67, 'S.O. Iced Coffee', 435), (63, 'Americano', 423), (62, 'Espresso', 364), (59, 'NOLA Carton', 337), (66, 'Chemex Coffee', 295), (64, 'Gibraltar', 248), (64, 'S.O. Iced - 4 oz Oji', 234), (60, 'Hot Chocolate', 210), (54, 'Cascara Fizz', 168), (62, 'Macchiato', 166), (60, 'Tea', 144), (69, 'Apple Juice', 130), (57, 'Sparkling Water', 127), (61, 'Au Lait, Unknown', 101), (59, 'Still Water', 61), (59, 'Milk', 43), (62, 'Juice', 23), (54, 'Shakerato', 21), (50, 'Kyoto Iced Coffee', 1), (50, 'Sowden Au Lait', 1)]


In [3]:
report_1 = pd.read_sql_query(report1_sql, conn)
report_1.columns = ['Temperature (Farenheit)','Item Name', 'Number Sold']
report_1

,Temperature (Farenheit),Item Name,Number Sold
0,56,Latte,1945
1,68,New Orleans Iced Coffee (cup),1913
2,61,Drip Coffee,1730
3,65,"Cappuccino, Unknown",1494
4,70,Mocha,444
5,67,S.O. Iced Coffee,435
6,63,Americano,423
7,62,Espresso,364
8,59,NOLA Carton,337
9,66,Chemex Coffee,295


In [4]:
####################################### Report 2 #############################################
create_view = '''create view if not exists analytics_view as  
                 select temperature_master.index_id,  temperature_master.date, 
                 temperature_master.avg_sf_temperature, sales_2016.item_name, sales_2016.net_quantity
                 from sales_2016 join temperature_master on sales_2016.sale_date = temperature_master.date
                 order by temperature_master.date'''

report2_sql = '''select warmer.item_name, warmer.avg_change_warmer, colder.avg_change_colder
                 from (
                 select result.item_name, avg(result.order_change) as avg_change_warmer  from (
                 select a1.date, a1.item_name, a1.net_quantity, (a2.avg_sf_temperature - a1.avg_sf_temperature) as sub1, (a2.net_quantity - a1.net_quantity) as order_change
                 from analytics_view a1, analytics_view a2
                 where a2.index_id - a1.index_id = 5) as result 
                 where result.sub1 = 2
                 group by result.item_name
                 order by avg_change_warmer desc) as warmer
                 join  (
                 select result.item_name, avg(result.order_change) as avg_change_colder from (
                 select a1.date, a1.item_name, a1.net_quantity, (a2.avg_sf_temperature - a1.avg_sf_temperature) as sub1, (a2.net_quantity - a1.net_quantity) as order_change
                 from analytics_view a1, analytics_view a2
                 where a2.index_id - a1.index_id = 5) as result 
                 where result.sub1 = -2
                 group by result.item_name
                 order by avg_change_colder  desc) as colder
                 on warmer.item_name = colder.item_name'''

c.execute(create_view)
c.execute(report2_sql)
print(c.fetchall())

[('Apple Juice', 0.33488372093023255, -0.0642570281124498), ('Au Lait, Unknown', 0.15384615384615385, 0.03233830845771144), ('Americano', 0.14666666666666667, 0.045081967213114756), ('NOLA Carton', 0.10772833723653395, -0.3674698795180723), ('Juice', 0.10752688172043011, 0.24324324324324326), ('Still Water', 0.1, 0.11940298507462686), ('Espresso', 0.07078313253012049, 0.025065963060686015), ('Tea', 0.06741573033707865, -0.010810810810810811), ('S.O. Iced - 4 oz Oji', 0.042028985507246375, 0.09669811320754718), ('Chemex Coffee', 0.020618556701030927, 0.12371134020618557), ('Drip Coffee', 0.01947015639961698, -0.07446808510638298), ('Latte', 0.01425, 0.006112909025530385), ('Cascara Fizz', 0.0045045045045045045, -0.14566929133858267), ('Sparkling Water', 0.002793296089385475, 0.06097560975609756), ('Mocha', -0.0032397408207343412, 0.10940919037199125), ('Gibraltar', -0.0072992700729927005, -0.005434782608695652), ('S.O. Iced Coffee', -0.014175257731958763, 0.11432926829268293), ('Cappucc

In [5]:
report_2 = pd.read_sql_query(report2_sql, conn)
report_2.columns = ['Item Name', 'Avg change in sales when colder', 'Avg change in sales when warmer']
report_2

,Item Name,Avg change in sales when colder,Avg change in sales when warmer
0,Apple Juice,0.334884,-0.064257
1,"Au Lait, Unknown",0.153846,0.032338
2,Americano,0.146667,0.045082
3,NOLA Carton,0.107728,-0.367470
4,Juice,0.107527,0.243243
5,Still Water,0.100000,0.119403
6,Espresso,0.070783,0.025066
7,Tea,0.067416,-0.010811
8,S.O. Iced - 4 oz Oji,0.042029,0.096698
9,Chemex Coffee,0.020619,0.123711


In [6]:
report_1.to_csv('report1.csv', index=False)
report_2.to_csv('report2.csv', index=False)